In [1]:
import torch

In [2]:
word2idx = {
    "e" : 0,
    "h": 1,
    "l": 2,
    "o": 3
}

idx2word = {}
for w in word2idx:
    idx2word[word2idx[w]] = w

def encode(seq):
    return [word2idx[w] for w in seq]

def decode(token):
    return [idx2word[i] for i in token]

def one_hot(idx, len = 4):
    hots = []
    for i in range(len):
        if(i == idx):
            hots.append(1)
        else:
            hots.append(0)
    return hots

def batch_one_hot(seq):
    token = encode(seq)
    return [one_hot(i) for i in token]


In [3]:
num_class = 4
input_size = 4
hidden_size = 8
embedding_size = 10
num_layers = 2
batch_size = 1
seq_len = 5

input_seq = "hello"
output_seq ="ohlol"
x_data = encode(input_seq)
y_data = encode(output_seq)

inputs = torch.tensor(x_data).reshape(batch_size, seq_len).long()
labels = torch.tensor(y_data).reshape(seq_len).long()
print(inputs)
print(labels)

tensor([[1, 0, 2, 2, 3]])
tensor([3, 1, 2, 3, 2])


In [4]:
class Model(torch.nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, num_class):
         super(Model, self).__init__()
         self.hidden_size = hidden_size
         self.num_layers = num_layers
         self.num_class = num_class
         self.emb = torch.nn.Embedding(input_size, embedding_size)
         self.rnn = torch.nn.GRU(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
         self.fc = torch.nn.Linear(hidden_size, num_class)

    def forward(self, x):
        hidden = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        x = self.emb(x)
        x, _ = self.rnn(x, hidden)
        x = self.fc(x)
        return x.view(-1, self.num_class)

net = Model(input_size, embedding_size, hidden_size, num_layers, num_class)

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)

In [6]:
for epoch in range(15):
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    print("Predicted string: ","".join(idx2word[x] for x in idx), end="")
    print(", Epoch [%d/15] loss=%.3f" % (epoch + 1, loss.item()))

Predicted string:  lllll, Epoch [1/15] loss=1.233
Predicted string:  lllll, Epoch [2/15] loss=1.100
Predicted string:  oolll, Epoch [3/15] loss=0.999
Predicted string:  oolll, Epoch [4/15] loss=0.893
Predicted string:  ohlll, Epoch [5/15] loss=0.804
Predicted string:  hhlll, Epoch [6/15] loss=0.726
Predicted string:  hhlll, Epoch [7/15] loss=0.640
Predicted string:  hhlll, Epoch [8/15] loss=0.552
Predicted string:  ohlol, Epoch [9/15] loss=0.458
Predicted string:  ohlol, Epoch [10/15] loss=0.354
Predicted string:  ohlol, Epoch [11/15] loss=0.248
Predicted string:  ohlol, Epoch [12/15] loss=0.164
Predicted string:  ohlol, Epoch [13/15] loss=0.109
Predicted string:  ohlol, Epoch [14/15] loss=0.073
Predicted string:  ohlol, Epoch [15/15] loss=0.049


In [7]:
def envalulate_word(model, input):
    
    with torch.no_grad():
        x_data = encode(input)
        inputs = torch.tensor(x_data).reshape(1, len(x_data)).long()
        outputs = model(inputs)
        _, idx = outputs.max(dim=1)
        idx = idx.data.numpy()
        print("Predicted string: ","".join(idx2word[x] for x in idx), end="")

In [8]:
envalulate_word(net, 'hello')

Predicted string:  ohlol